In [1]:
import numpy as np
import scipy.special as sp
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.spatial import ConvexHull
from shapely.geometry import Polygon
from shapely.ops import unary_union

# Parameters
n_pts = 5000
min_log10 = -6
max_log10 = 6
n_dim = 4
koff_r2w = 1e3
colors = ['#f0f921', '#e16462', '#b12a90', '#6a00a8']
numbers = [4, 3, 2, 1]

# Optimized tradeoff_reaction_expression_n function
def tradeoff_reaction_expression_n(kvec, koff_r2w, n):
    koff, kcat, kon, kp = kvec
    koffw = koff * koff_r2w
    part1_R = (koff + kp)**n
    part1_W = (koffw + kp)**n
    sum_terms = 0
    for i in range(n):
        coefficient = sp.comb(n-1, i, exact=True) * (i + 1)
        term = coefficient * koff**(n-1 - i) * kp**i
        sum_terms += term
    part2_R = kon * sum_terms
    part2_W = kon * sum_terms
    activity = kon * np.power(kp, n) / ((koff + kon) * np.power((koff + kp), n) + kcat * (part1_R + part2_R))
    activity_flux = activity * kcat
    fidelity = ((koffw + kon) * np.power((koffw + kp), n) + kcat * (part1_W + part2_W)) / \
               ((koff + kon) * np.power((koff + kp), n) + kcat * (part1_R + part2_R))
    fidelity_normalized = fidelity / koff_r2w**(n + 1)
    return [activity, fidelity, activity_flux, fidelity_normalized, koff, kcat, kon, kp]

# tradeoff_result_n function
def tradeoff_result_n(kk, koff_r2w, n):
    result_n_x = np.zeros((n_pts, 8))
    for i in range(n_pts):
        result_n_x[i] = tradeoff_reaction_expression_n(kk[i, :], koff_r2w, n)
    return result_n_x

# Set Seaborn context
sns.set_context('talk', font_scale=2, rc={'lines.linewidth': 3})



In [2]:
def generate_points_for_ratio(kp_koff_ratio):
    """
    Generate points with a fixed kp/koff ratio.

    Parameters:
    - n_pts: Number of points to generate.
    - n_dim: Number of dimensions (expected >= 4 for koff and kp).
    - min_log10: Minimum log10 value for random sampling.
    - max_log10: Maximum log10 value for random sampling.
    - kp_koff_ratio: Desired kp/koff ratio (e.g., 0.01, 0.1, 1, 10, 100).

    Returns:
    - Array of shape (n_pts, n_dim) with specified kp/koff ratio.
    """
    # Initialize array
    points = np.zeros((n_pts, n_dim))
    
    # Generate first three columns (koff and others) in log scale
    points[:, 0:3] = 10 ** (np.random.rand(n_pts, 3) * (max_log10 - min_log10) + min_log10)
    
    # Set kp (column 3) based on koff (column 0) and the desired ratio
    points[:, 3] = points[:, 0] * kp_koff_ratio
    
    # Verify the ratio
    assert np.allclose(points[:, 3] / points[:, 0], kp_koff_ratio, rtol=1e-5), \
        f"kp/koff ratio is not {kp_koff_ratio}"
    
    return points

In [1]:
# Define desired kp/koff ratios
ratios = [1]  # Corresponding to 1:100, 1:10, 1:1, 10:1, 100:1

# Generate datasets for each ratio
datasets = {}
for ratio in ratios:
    dataset_name = f"kk_ratio_{ratio}"
    datasets[dataset_name] = generate_points_for_ratio(ratio)

#print (datasets[f"kk_ratio_0.01"])

import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
import seaborn as sns

# Set seaborn style
sns.set_context('talk', font_scale=1.8, rc={'lines.linewidth': 2})

# Aggregate all points from different n values
def get_all_points_for_hull(kk, koff_r2w, n_list):
    points = []
    for n in n_list:
        points.append([])
        result = np.array([tradeoff_reaction_expression_n(kvec, koff_r2w, n) for kvec in kk])
        # Use fidelity (x) and activity (y)
        fidelity = result[:, 1]
        activity = result[:, 0]
        # Filter out any invalid or zero values to avoid log-scaling issues
        valid = (fidelity > 0) & (activity > 0)
        log_points = np.log10(np.vstack([fidelity[valid], activity[valid]]).T)
        points[-1].append(log_points)
    return np.vstack(points) 

# Example usage with kk_less
kk_ratio_log_points = {}
for ratio in ratios:
    kk_ratio_log_points_name = f"{ratio}"
    kk_ratio_log_points [kk_ratio_log_points_name]= get_all_points_for_hull(datasets[f"kk_ratio_{ratio}"], koff_r2w, numbers)
 

NameError: name 'generate_points_for_ratio' is not defined

In [4]:
def create_plot_convex(all_log_points, ax, label, color):
    hulls = []
    polys = []

    for i in range(len(numbers)):
        log_pts = all_log_points[i]
        if log_pts.shape[0] < 3:
            continue  # Skip if not enough points

        try:
            hull = ConvexHull(log_pts)
            hulls.append(hull)
            polys.append(Polygon(log_pts[hull.vertices]))
        except:
            continue  # Skip problematic sets

    if polys:
        union_poly = unary_union(polys)
        if not union_poly.is_empty:
            x, y = union_poly.exterior.xy
            x = 10**np.array(x)
            y = 10**np.array(y)
            ax.plot(x, y, alpha=1, color=color, lw=5, label=label)


# Function to create a single plot with colors by n
def create_plot(kk, title, ax, max_points=1000):
    for idx, i in enumerate(numbers):
        result_n_i = tradeoff_result_n(kk, koff_r2w, i) 

        # Randomly select up to `max_points`
        n_total = result_n_i.shape[0]
        if n_total > max_points:
            idx_selected = np.random.choice(n_total, max_points, replace=False)
            result_n_i_selected = result_n_i[idx_selected]

        ax.scatter(result_n_i_selected[:, 1], result_n_i_selected[:, 0], s=20, color=colors[idx % len(colors)], label=f'n = {i}')
        xx = np.logspace(-20, 0, 1000)
        ax.plot((1/xx**(1/(i+1))-1)**(i+1)*xx*(1000**(i+1)), xx, lw=3, color=colors[idx % len(colors)],
                linestyle='--', alpha=1)

In [ ]:

fig1, ax1 = plt.subplots(1, 1, figsize=(7, 7))

for idx, ratio in enumerate(ratios):
    create_plot(
        kk_ratio_log_points[f"{ratio}"],
        ax1,
        rf'$k_p/k_{{\text{{off}}}} = {ratio}$',
        color=colors[idx % len(colors)]
    )

ax1.set_ylabel(r'$c$ activity')
ax1.set_xlabel(r'$\eta$ fidelity')
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlim([1, 1e13])
ax1.set_ylim([1e-3, 2])
ax1.legend(bbox_to_anchor=(0, 1.2), loc='lower left')
plt.tight_layout()
plt.show()


: 